# Считывание данных из Google Sheet

Набор данных оформлен [в виде электронной таблицы на Google Drive](https://docs.google.com/spreadsheets/d/1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI/edit?usp=sharing). Доступ к Google Sheets API осуществляется в соответствии с [документацией API для Python](https://developers.google.com/sheets/api/quickstart/python).

Импортируем данные учетной записи и преобразуем в объект Pandas:

In [25]:
# встроенныеimport json
import os.path
import pickle       # хранение токенов
import collections  # используется для подсчета
import re           # обработка текстовой составляющей

import pandas as pd
    
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI' # id гугл-таблицы
RANGE_NAME = 'Memories' # Забираем лист целиком диапазон

In [26]:
def get_data():
    """Авторизует и забирает данные из Google SpreadSheet"""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    
    return values


def get_df():
    data = get_data()
    df = pd.DataFrame.from_records(data)
    headers = df.iloc[0]  # Названия столбцов содержатся в нулевой строке таблицы
    df = pd.DataFrame(df.values[1:], columns=headers)
    df = df.set_index('№') # Номер строки соответствует номеру воспоминания
    
    # Добавим столбец подсчета числа символов, из которых состоит запись.
    df ['Число символов'] = df['Воспоминание'].apply(lambda x: len(x))
    return df
    
df = get_df()

In [38]:
from datetime import datetime
from datetime import timedelta

2020-01-16


In [27]:
not_published = [75, 2, 228, 159, 173, 106, 72, 181, 10, 59, 56, 87, 39, 242,
 54, 151, 76, 204, 28, 180, 64, 183, 239, 135, 168, 65, 199,
 202, 188, 6, 70, 71, 177, 163, 33, 187, 91, 19, 165, 223,
 44, 192, 142, 224, 227, 171, 184, 45, 125, 250, 47, 97, 214,
 83, 148, 164, 61, 43, 161, 185, 111, 1, 205, 169, 35, 182, 197,
 251, 7, 132, 221, 246, 138, 8, 17, 156, 155, 129, 172, 252, 11,
 201, 253, 40, 48, 232, 189, 174, 217, 210, 55, 116, 90, 233, 108,
 112, 157, 5, 254, 117, 85, 60, 208, 4, 68, 248, 38, 113, 51, 237,
 170, 26, 213, 147, 96, 21, 209, 215, 216, 63, 69, 81, 200, 194,
 88, 20, 143, 126, 158, 219, 231, 84, 89, 31, 144, 121, 94, 193,
 222, 99, 29, 92, 37, 235, 128, 191, 42, 93, 114, 123, 119, 78, 3,
 100, 186, 86, 104, 120, 245, 74, 13, 115, 229, 241, 77, 73, 18, 220,
 34, 12, 162, 107, 149, 57, 225, 130, 46, 206, 166, 118, 218, 53, 247,
 22, 179, 134, 139, 196, 152, 79, 16, 226, 30, 52, 49, 236, 124, 25, 95,
 175, 195, 15, 110, 103, 141, 27, 207, 154, 80, 82, 244, 14, 230, 58, 137,
 240, 98, 109, 167, 131, 238, 234]

In [59]:
rus = '{0}:\n\n«{1}».\n\n#воспоминание {2}, #непереводное\n'
eng = '{0}:\n\n"{1}".\n\n#memory {2}, #translated #rus\n'

for i, item in enumerate(not_published):
    name_rus = df.loc[str(item)]['Имя']
    if name_rus == '-':
        name_rus = 'Без имени'
    name_eng = df.loc[str(item)]['Name']
    if name_eng == '-':
        name_eng = 'No name'
    
    memory_rus = df.loc[str(item)]['Воспоминание'].strip()
    if memory_rus[-1] == '.':
        memory_rus = memory_rus[:-1]
        
    memory_eng = df.loc[str(item)]['Translation'].strip()
    if memory_eng[-1] == '.':
        memory_eng = memory_eng[:-1]
    
    d = (datetime.today()+timedelta(days=1+i)).date()
    
    print("===== {} ====".format(d))
    print(rus.format(name_rus, memory_rus, item))
    print(eng.format(name_eng, memory_eng, item))

===== 2020-01-16 ====
Никита:

«Помню, как мне в детстве удаляли распухший лимфоузел, как я был в больнице. Мне было года три. Вообще помню это время в три-четыре года хорошо. Я тогда учился читать. Помню, как осознанно прятался от родителей. Они не понимали, где я, искали, переживали, а я видел это и смеялся над ними».

#воспоминание 75, #непереводное

Nikita :

"Remember myself as a child when I got a swollen lymph node removed in the hospital. I was three. I do remember that time in three or four years. I learnt to read. I remember how deliberately I was hiding from parents.
They didn't know where I was, were looking for, were worried, and I saw it and laughed at them".

#memory 75, #translated #rus

===== 2020-01-17 ====
Без имени:

«Мне года полтора. Я беру маленький деревянный стульчик и тащу его в комнату из коридора. В комнате двери со стеклянными квадратными вставками, на стены падает солнечный свет».

#воспоминание 2, #непереводное

No name:

"I am about a year and a half. I 

IndexError: string index out of range